to solve this question https://stackoverflow.com/questions/24612626/b-spline-interpolation-with-python/24693358#24693358

In [1]:
import os

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [3]:
import numpy as np
import scipy.interpolate as si

b spline interpolation from https://stackoverflow.com/questions/24612626/b-spline-interpolation-with-python

In [4]:
#curve base
points = [[-6, 2], [-3, -2],[0, 0], [0, 2], [2, 3], [4, 0], [6, 3], [8, 5], [8, 8], [6, 8], [5, 9], [7, 2]];
points = np.array(points)
x = points[:,0]
y = points[:,1]


t = range(len(points))
ipl_t = np.linspace(0.0, len(points) - 1, 100)


x_tup = si.splrep(t, x, k=3)
y_tup = si.splrep(t, y, k=3)

x_list = list(x_tup)
xl = x.tolist()
x_list[1] = xl + [0.0, 0.0, 0.0, 0.0]

y_list = list(y_tup)
yl = y.tolist()
y_list[1] = yl + [0.0, 0.0, 0.0, 0.0]

x_i = si.splev(ipl_t, x_list)
y_i = si.splev(ipl_t, y_list)

In [7]:
url = "https://cgm.demo-de.cloud.c3.cgm.ag/cgmg3"
service = Service(executable_path='./chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

In [8]:
action =  ActionChains(driver);
startElement = driver.find_element(By.ID, '287e41c0-a5de-4102-8d17-5c247b015e9b')
#First, go to your start point or Element
action.move_to_element(startElement);
action.perform();

for mouse_x, mouse_y in zip(x_i, y_i):
    action.move_by_offset(mouse_x,mouse_y);
    action.perform();
    sleep(0.1)
    print(mouse_x, mouse_y)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="287e41c0-a5de-4102-8d17-5c247b015e9b"]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6D3A7E1F5+2972373]
	(No symbol) [0x00007FF6D37155F0]
	(No symbol) [0x00007FF6D35B57FA]
	(No symbol) [0x00007FF6D3605A3E]
	(No symbol) [0x00007FF6D3605D2C]
	(No symbol) [0x00007FF6D364EAB7]
	(No symbol) [0x00007FF6D362BABF]
	(No symbol) [0x00007FF6D364B8CC]
	(No symbol) [0x00007FF6D362B823]
	(No symbol) [0x00007FF6D35F75E8]
	(No symbol) [0x00007FF6D35F8751]
	GetHandleVerifier [0x00007FF6D3AA47BD+3129501]
	GetHandleVerifier [0x00007FF6D3AF4D00+3458528]
	GetHandleVerifier [0x00007FF6D3AEB05D+3418429]
	GetHandleVerifier [0x00007FF6D387687B+844123]
	(No symbol) [0x00007FF6D3720AFF]
	(No symbol) [0x00007FF6D371C6D